In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**1. Formulate this as one of the discrete optimization problems you learnt in IE 684. Clearly describe the formulation with explicitly listing the variables, objective function and constraints**

Let xi represent the number of boxes of type i produced, where xi is a non-negative integer xi ∈ ℕ ∪ {0}.

Let yi be a binary variable indicating whether any boxes of type i are produced. yi = 1 if xi > 0, and yi = 0 otherwise.

This constraint can be enforced using the big M method, meaning:

  xi ≤ M * yi    for i = 1, 2, ..., 6
Where M is sufficiently large.

Let vi be the volume of box type i. Then the objective function becomes:

  minimize   ∑(i=1 to 6) (1500 * yi + 100 * vi * xi)

The constraints, including the big M method:

1. The demand should be met:
   xi ≥ di    for all i = 1, 2, ..., 6
Where di is the demand for box type i.

2. xi ≤ M * yi    for i = 1, 2, ..., 6


**2. Solve it using an algorithm you have used previously in IE 684 Lab. List a table of the optimal number of boxes
of each size and state the optimal cost. Don’t use solver.**

In [ ]:
# Define the demand, volume, and selling price of each box
boxes = [
    {"demand": 300, "volume": 0.15, "price": 300},
    {"demand": 450, "volume": 0.25, "price": 300},
    {"demand": 80, "volume": 0.35, "price": 350},
    {"demand": 500, "volume": 0.5, "price": 500},
    {"demand": 150, "volume": 0.6, "price": 600},
    {"demand": 800, "volume": 0.75, "price": 800}
]

# Sort the boxes by volume in descending order
sorted_boxes = sorted(boxes, key=lambda x: x["volume"], reverse=True)

# Initialize variables
produced_boxes = [0, 0, 0, 0, 0, 0]  # Number of boxes produced for each size
total_cost = 0

# Greedy algorithm to produce boxes
for box in sorted_boxes:
    index = boxes.index(box)
    demand = box["demand"]
    volume = box["volume"]
    variable_cost = 100 * volume
    fixed_cost = 1500
    available_demand = min(demand, produced_boxes[index - 1])  # Use boxes of larger size to satisfy demand
    produced_boxes[index] = demand - available_demand  # Produce remaining boxes to meet demand
    total_cost += (variable_cost * produced_boxes[index]) + (fixed_cost * produced_boxes[index])

# Output the optimal number of boxes and total cost
print("Optimal Number of Boxes Produced for Each Size:", produced_boxes)
print("Optimal Total Cost:", total_cost)


Optimal Number of Boxes Produced for Each Size: [0, 450, 80, 500, 150, 800]
Optimal Total Cost: 3078050.0


**3. Suppose only 1000 cubic meter of space is available for transport for Flipbakox company. So they may not be able
to fulfill whole demand. Instead they would want to maximize their profit. Formulate this as an optimization
problem. Clearly describe the formulation with explicitly listing the variables, objective function and constraints.**

Let xi represent the number of boxes of type i produced, where xi is a non-negative integer xi ∈ ℕ ∪ {0}.

Let yi be a binary variable denoting the production status of box type i, where yi = 1 if xi > 0, and yi = 0 otherwise.

The price of product xi is denoted by pi. The volume of box type i is represented by vi.

To enforce the production constraints using the big M method, the following inequality applies:
  xi ≤ M * yi    for i = 1, 2, ..., 6
where M is a sufficiently large constant.

The objective function aims to maximize profit, considering the selling price and the cost of manufacturing:

  maximize   ∑(i=1 to 6) (pi * xi - (1500 * yi + 100 * vi * xi))

Under the following constraints:

1. The total volume of produced boxes must not exceed 1000:
   ∑(j=1 to 6) (xj * vj) ≤ 1000

2. The big M method constraint for each type of box:
   xi ≤ M * yi    for i = 1, 2, ..., 6


**4. Solve the problem. You can use a solver. Write, explain and interpret the results. Your answer should clearly
explain how many boxes of each type is required to be transported.**

In [ ]:
boxes = [[1, 300, 0.15, 300],
  [2,450,0.25,300],
  [3 ,80 ,0.35, 350],
  [4 ,500 ,0.5 ,500],
  [5 ,150 ,0.6, 600],
  [6 ,800 ,0.75 ,800]]
demand_in_vol = []
box_no = []
vol_per_box  = []
price = []
demand = []
for ele in boxes:
  demand_in_vol.append(ele[1]*ele[2])
  box_no.append(ele[0])
  vol_per_box.append(ele[2])
  price.append(100*ele[2]+ele[3]+1500)
  demand.append(ele[1])

In [ ]:
price

[1815.0, 1825.0, 1885.0, 2050.0, 2160.0, 2375.0]

In [ ]:
model = ConcreteModel()
model.x = Var(range(6), domain = NonNegativeIntegers)
model.y = Var(range(6), domain = Binary)
model.obj = Objective(expr = sum(( (price[i] - 100*volume[i])*model.x[i] - 1500*model.y[i] )for i in range(6)), sense = maximize)

model.constraints = ConstraintList()
model.constraints.add(expr = sum(model.x[i]*volume[i] for i in range(6)) <= 1000)
for i in range(6):
  model.constraints.add(expr = model.x[i] <= M*model.y[i])

result = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)
model.pprint()

In [ ]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [ ]:
from pyomo.environ import *

In [ ]:
boxes = [[1, 300, 0.15, 300],
  [2,450,0.25,300],
  [3 ,80 ,0.35, 350],
  [4 ,500 ,0.5 ,500],
  [5 ,150 ,0.6, 600],
  [6 ,800 ,0.75 ,800]]


In [ ]:
demand_in_vol = []
box_no = []
vol_per_box  = []
price = []

for ele in boxes:
  demand_in_vol.append(ele[1]*ele[2])
  box_no.append(ele[0])
  vol_per_box.append(ele[2])
  price.append(100*ele[2]+ele[3]+1500)

In [ ]:
demand_in_vol

[45.0, 112.5, 28.0, 250.0, 90.0, 600.0]

In [ ]:
model = ConcreteModel()
model.y = Var(range(6), domain = NonNegativeIntegers)
model.obj = Objective(expr = sum(model.y[i]*price[i] for i in range(6)), sense = minimize)

model.constraints = ConstraintList()
for i in range(6):
          model.constraints.add(expr = sum(model.y[i]*vol_per_box[i] for i in range(6)) >= sum(demand_in_vol))

result = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

In [ ]:
model.obj()

1238050.0

In [ ]:
print('Decision Variables for our model : ')
for i in range(6):
    print(f'x{i} : ', model.y[i].value)

Decision Variables for our model : 
x0 :  0.0
x1 :  0.0
x2 :  0.0
x3 :  1.0
x4 :  1875.0
x5 :  0.0
